LABELED DATA

In [2]:
from datasets import load_dataset

data = load_dataset("cornell-movie-review-data/rotten_tomatoes")

c:\Users\SHREYAS\OneDrive\Desktop\LLMs\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# pip install datasets

In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [5]:
data["train"][0, -1]

{'text': ['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
  'things really get weird , though not particularly scary : the movie is all portent and no content .'],
 'label': [1, 0]}

In [6]:
from transformers import pipeline

#path to model
model_path="cardiffnlp/twitter-roberta-base-sentiment-latest"

pipe= pipeline(
    model=model_path,
    tokenizer=model_path,
    return_all_scores=True,
    device="cuda:0"
)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
c:\Users\SHREYAS\OneDrive\Desktop\LLMs\venv\Lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k

In [7]:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


In [8]:
#test the split part

import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

y_pred=[]
for output in tqdm(pipe(KeyDataset(data["test"], "text")), total=len(data["test"])):                    
                    negative_score=output[0]["score"]
                    positive_score=output[2]["score"]
                    assignment = np.argmax([negative_score, positive_score])
                    y_pred.append(assignment)

100%|██████████| 1066/1066 [00:15<00:00, 69.56it/s]


In [9]:
#pip install transformers


In [10]:
from sklearn.metrics import classification_report
def evaluation(y_true, ypred):

    performance= classification_report(
        y_true, y_pred,
        target_names=["Negative Review", "Positive Review"]
    )
    print(performance)

In [11]:
evaluation(data["test"]["label"], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.76      0.88      0.81       533
Positive Review       0.86      0.72      0.78       533

       accuracy                           0.80      1066
      macro avg       0.81      0.80      0.80      1066
   weighted avg       0.81      0.80      0.80      1066



USING GENERATIVE MODELS

In [12]:
pipe= pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    device="cuda:0"
)


Device set to use cuda:0


In [13]:
#prepare our data

prompt= "Is the following sentence positve or negative?"

data= data.map(lambda example: {"t5": prompt+example["text"]})
data

Map: 100%|██████████| 1066/1066 [00:00<00:00, 29190.82 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 1066
    })
})

In [14]:
#run inference

y_pred=[]
for output in tqdm(pipe(KeyDataset(data["test"], "t5")), total=len(data["test"])):
    text = output[0]["generated_text"]
    y_pred.append(0 if text == "negative" else 1)

100%|██████████| 1066/1066 [00:46<00:00, 23.10it/s]


In [15]:
evaluation(data["test"]["label"], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.84      0.84      0.84       533
Positive Review       0.84      0.84      0.84       533

       accuracy                           0.84      1066
      macro avg       0.84      0.84      0.84      1066
   weighted avg       0.84      0.84      0.84      1066

